<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
!pip install webdriver_manager

  Obtaining dependency information for webdriver_manager from https://files.pythonhosted.org/packages/b1/51/b5c11cf739ac4eecde611794a0ec9df420d0239d51e73bc19eb44f02b48b/webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata
  Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata (12 kB)
Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl (27 kB)


In [4]:
# 모듈 import
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import time 

In [1]:
target_categories = ['게요리', '곱창,막창,양', '과일,주스전문점', '국밥', '국수', '김밥', '낙지요리', '냉면', 
    '다이어트,샐러드', '닭갈비', '닭발', '도시락,컵밥', '돈가스', '돼지고기구이', '두부요리', 
    '딤섬,중식만두', '떡볶이', '만두', '매운탕,해물탕', '맥주,호프', '바(BAR)', '베이글', '베이커리', 
    '베트남음식', '북카페', '분식', '브런치카페', '샌드위치', '생선회', '샤브샤브', '소고기구이', 
    '스테이크,립', '스파게티,파스타전문', '아시아음식', '양갈비', '양꼬치', '양식', '오뎅,꼬치', 
    '와인', '와플', '요리주점', '우동,소바', '육류,고기요리', '이자카야', '이탈리아음식', '인도음식', 
    '일식당', '일식튀김,꼬치', '장어,먹장어요리', '조개요리', '족발,보쌈', '종합분식', '주꾸미요리', 
    '중식당', '찌개,전골', '초밥,롤', '치킨,닭강정', '카페', '카페,디저트', '칼국수,만두', '케이크전문', 
    '태국음식', '테이크아웃커피', '토스트', '포장마차', '퓨전음식', '프랑스음식', '피자', '한식', 
    '한식뷔페', '한정식-일반', '해물,생선요리', '해장국', '햄버거', '감자탕', '곰탕,설렁탕', '기사식당', 
    '덮밥', '라면', '문래돼지불백', '백반,가정식', '백숙,삼계탕', '복어요리', '빙수', '순대,순댓국', 
    '아이스크림', '오므라이스', '죽', '찜닭', '추어탕', '카레', '패밀리레스토랑', '향토음식', 
    '홍차전문점', '후렌치후라이', '호떡', '쌈밥', '아귀찜,해물찜', '닭볶음탕', '대게요리', '마라탕', 
    '멕시코,남미음식', '브런치', '스페인음식', '정육식당', '전통,민속주점', '일본식라면']

In [2]:
len(target_categories)

107

In [7]:
# 각자 동에 맞는 거로!

restaurant = pd.read_csv('서울시서대문구일반음식점인허가정보.csv', encoding = 'cp949') 
restaurant = restaurant[restaurant['영업상태명'] == '영업/정상']

# NaN 값을 빈 문자열로 대체, 안 하면 에러남
restaurant['지번주소'] = restaurant['지번주소'].fillna('')

# '지번주소' 열이 '서울특별시 마포구 신수동'으로 시작하는 행만 선택
target_dong = '서울특별시 서대문구 창천동'
dong = restaurant[restaurant['지번주소'].str.startswith(target_dong)]

# dong에서 도로명주소를 기준으로 검색할 것이기 때문에, 주소 기준 중복인 행을 다 없앰

dong = dong.drop_duplicates(subset='도로명주소', keep=False)
dong.reset_index(drop=True, inplace=True)
dong

C:\Users\uje10\AppData\Local\Temp\ipykernel_6640\1709106719.py:3: DtypeWarning: Columns (13,28,29,30,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  restaurant = pd.read_csv('서울시서대문구일반음식점인허가정보.csv', encoding = 'cp949')


,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,3120000,3120000-101-1971-05561,19710811,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,57.87,NaN,NaN,NaN
1,3120000,3120000-101-1977-00201,19770808,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,19.80,NaN,NaN,NaN
2,3120000,3120000-101-1977-00342,19770803,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,67.00,NaN,NaN,NaN
3,3120000,3120000-101-1977-01871,19770609,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3120000,3120000-101-1978-09255,19781220,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Y,357.17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,3120000,3120000-101-2024-00134,2024-04-26,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
640,3120000,3120000-101-2024-00141,2024-05-01,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641,3120000,3120000-101-2024-00145,2024-05-02,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,3120000,3120000-101-2024-00156,2024-05-13,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# webdriver 실행
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 검색 결과가 표시되는 iframe으로 전환
def search_iframe():
    driver.switch_to.default_content()  # 차상위 (기본) 프레임으로 전환
    try:
        time.sleep(2)  # 페이지 로딩 대기
        driver.switch_to.frame("searchIframe")  # 검색 결과 프레임으로 전환
    except NoSuchElementException:
        print("searchIframe을 찾을 수 없습니다.")
        return False
    return True

# 개별 업체의 세부 정보가 표시되는 iframe으로 전환
def entry_iframe():
    driver.switch_to.default_content()  # 차상위 (기본) 프레임으로 전환
    try:
        time.sleep(2)  # 페이지 로딩 대기
        driver.switch_to.frame("entryIframe")  # 세부 정보 프레임으로 전환
    except NoSuchElementException:
        print("entryIframe을 찾을 수 없습니다.")
        return False
    return True

# type1의 화면에 표시된 업체명을 확인하는 함수
def chk_names_type1():
    search_iframe()  # search 프레임으로 전환
    elems = driver.find_elements(By.CLASS_NAME, 'place_bluelink.C6RjW')
    name_list = []
    category_list = []
    elements = []
    
    for e in elems:
        try:
            name = e.find_element(By.CLASS_NAME, 'YwYLL').text
            category = e.find_element(By.CLASS_NAME, 'YzBgS').text
            
            # 업종이 target_categories에 포함되지 않으면 패스
            if category not in target_categories:
                continue
                
            name_list.append(name)
            category_list.append(category)
            elements.append(e)
        except:
            continue
            
    return elements, name_list, category_list

# type2의 화면에 표시된 업체명을 확인하는 함수
def chk_names_type2(processed_names):
    elems = driver.find_elements(By.CLASS_NAME, 'search_box')
    name_list = []
    category_list = []
    elements = []
    
    for e in elems:
        try:
            name = e.find_element(By.CLASS_NAME, 'search_title').text
            category = e.find_element(By.CLASS_NAME, 'category').text
            
            # 이미 처리된 이름은 패스
            if name in processed_names:
                continue
            
            # 업종이 target_categories에 포함되지 않으면 패스
            if category not in target_categories:
                continue
                
            name_list.append(name)
            category_list.append(category)
            elements.append(e)
        except:
            continue
            
    return elements, name_list, category_list

# 세부 페이지에서 URL과 ID를 추출하는 함수 (type1)
def chk_details():
    if not entry_iframe():  # 세부 정보의 iframe으로 전환
        return float('nan'), float('nan')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        # 특정 meta 태그를 찾기
        meta_tag = soup.find('meta', {'id': 'og:url'})

        # URL에서 숫자 부분만 추출
        if meta_tag:
            url = meta_tag['content']
            ID = url.split('/')[4]
    except:
        url = float('nan')
        ID = float('nan')

    search_iframe()  # 다시 검색 결과의 iframe으로 전환
    return url, ID

# 세부 페이지에서 URL과 ID를 추출하는 함수 (type2)
def chk_details_type2():
    if not entry_iframe():  # 세부 정보의 iframe으로 전환
        return float('nan'), float('nan')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        # 특정 meta 태그를 찾기
        meta_tag = soup.find('meta', {'id': 'og:url'})

        # URL에서 숫자 부분만 추출
        if meta_tag:
            url = meta_tag['content']
            ID = url.split('/')[4]
    except:
        url = float('nan')
        ID = float('nan')

    return url, ID

# 크롤링 메인 함수
def crawling_main(type):
    global naver_res  # 전역변수 naver_res 사용 (크롤링 데이터를 저장하는 데이터프레임)
    global target_dong
    
    processed_names = set()  # 이미 처리된 업체명을 추적하기 위한 집합
    
    if type == "type1":
        elements, name_list, category_list = chk_names_type1()
        details_func = chk_details
        
        id_list = []
        url_list = []

        for e in elements:
            try:
                e.click()  # 업체명 클릭
                time.sleep(2)  # 페이지 로딩 대기
                url, ID = details_func()  # 업체의 세부 정보 가져오기
                url_list.append(url)
                id_list.append(ID)

            except:
                url_list.append(float('nan'))
                id_list.append(float('nan'))
                continue
        
        # 새로운 데이터프레임 생성
        naver_temp = pd.DataFrame({
            '지번주소': [current_jibun] * len(name_list),
            '도로명주소': [current_keyword] * len(name_list),
            '업체명': name_list,
            '업종': category_list,
            'URL': url_list,
            'ID': id_list
        })
        
        # 기존 데이터프레임과 결합
        naver_res = pd.concat([naver_res, naver_temp])
        naver_res.reset_index(drop=True, inplace=True)
        naver_res.to_csv(f'./{target_dong.split()[-1]}.csv')  # 결과를 csv 파일로 저장
        
    if type == "type2":
        all_name_list = []
        all_category_list = []
        all_id_list = []
        all_url_list = []
        
        while True:
            elements, name_list, category_list = chk_names_type2(processed_names)
            details_func = chk_details_type2

            if len(elements) == 0:
                break

            for e, name, category in zip(elements, name_list, category_list):
                try:
                    e.click()  # 업체명 클릭
                    time.sleep(2)  # 페이지 로딩 대기
                    url, ID = details_func()  # 업체의 세부 정보 가져오기
                    all_url_list.append(url)
                    all_id_list.append(ID)
                    processed_names.add(name)  # 처리된 이름을 추가
                    all_name_list.append(name)
                    all_category_list.append(category)
                    driver.back()  # 뒤로가기 버튼 클릭
                    time.sleep(2)  # 뒤로가기 후 페이지 로딩 대기
                except:
                    continue
        
        # 새로운 데이터프레임 생성
        naver_temp = pd.DataFrame({
            '지번주소': [current_jibun] * len(all_name_list),
            '도로명주소': [current_keyword] * len(all_name_list),
            '업체명': all_name_list,
            '업종': all_category_list,
            'URL': all_url_list,
            'ID': all_id_list
        })

        # 기존 데이터프레임과 결합
        naver_res = pd.concat([naver_res, naver_temp])
        naver_res.to_csv(f'./{target_dong.split()[-1]}.csv')  # 결과를 csv 파일로 저장

# 불러올 정보를 담을 빈 데이터프레임 생성
naver_res = pd.DataFrame(columns=['지번주소', '도로명주소', '업체명', '업종', 'URL', 'ID'])

for index, row in dong.iterrows():
    current_jibun = row['지번주소']  # '지번주소'를 저장
    current_keyword = row['도로명주소']  # '도로명주소'를 검색 키워드로 사용
    
    print(f'전체 {len(dong)} 중 {index+1}, 주소:{current_jibun}')
    
    url = f'https://map.naver.com/p/search/{current_keyword}'
    driver.get(url)

    # "sc-1wsjitl dunggE overlap" 클래스가 있는지 확인
    try:
        time.sleep(2)  # 페이지 로딩 대기
        overlap_element = driver.find_element(By.CLASS_NAME, 'sc-1wsjitl.dunggE.overlap')
        try:
            more_button = overlap_element.find_element(By.CLASS_NAME, 'link_more')
            more_button.click()
            time.sleep(2)  # "더보기" 버튼 클릭 후 페이지 로딩 대기
        except NoSuchElementException:
            pass  # "더보기" 버튼이 없는 경우 그냥 넘어감
        crawling_main("type2")
    except NoSuchElementException:
        try:
            no_result = driver.find_element(By.CLASS_NAME, 'correction_result_text')
            continue
        except:
            try:
                crawling_main("type1")
            except NoSuchElementException:
                continue

    action = ActionChains(driver)
driver.quit()

전체 644 중 1, 주소:서울특별시 서대문구 창천동 30-10번지 
전체 644 중 2, 주소:서울특별시 서대문구 창천동 13-26번지 1층 
전체 644 중 3, 주소:서울특별시 서대문구 창천동 18-11번지 (지상1층) 
전체 644 중 4, 주소:서울특별시 서대문구 창천동 30-20
전체 644 중 5, 주소:서울특별시 서대문구 창천동 62-11번지 (지상2,3층) 
전체 644 중 6, 주소:서울특별시 서대문구 창천동 31-49
전체 644 중 7, 주소:서울특별시 서대문구 창천동 13-26번지 
전체 644 중 8, 주소:서울특별시 서대문구 창천동 13-33 1층 
전체 644 중 9, 주소:서울특별시 서대문구 창천동 2-80번지 
전체 644 중 10, 주소:서울특별시 서대문구 창천동 2-44번지 ,45 
전체 644 중 11, 주소:서울특별시 서대문구 창천동 5-46번지 
전체 644 중 12, 주소:서울특별시 서대문구 창천동 13-28번지 
전체 644 중 13, 주소:서울특별시 서대문구 창천동 5-47 
전체 644 중 14, 주소:서울특별시 서대문구 창천동 13-20 ,13-109, 13-114 (지하1층, 지상2,3층) 
전체 644 중 15, 주소:서울특별시 서대문구 창천동 4-77 79 (지상1층)
전체 644 중 16, 주소:서울특별시 서대문구 창천동 9-20번지 (지상1층) 
전체 644 중 17, 주소:서울특별시 서대문구 창천동 52-152 
전체 644 중 18, 주소:서울특별시 서대문구 창천동 31-17
전체 644 중 19, 주소:서울특별시 서대문구 창천동 52-5번지 6,7 
전체 644 중 20, 주소:서울특별시 서대문구 창천동 52-43
전체 644 중 21, 주소:서울특별시 서대문구 창천동 18-10
전체 644 중 22, 주소:서울특별시 서대문구 창천동 52-43번지 3층 
전체 644 중 23, 주소:서울특별시 서대문구 창천동 2-23번지 
전체 644 중 24, 주소:서울특별시 서대문구 창천동 52-123
전체 

전체 644 중 195, 주소:서울특별시 서대문구 창천동 4-1번지 
전체 644 중 196, 주소:서울특별시 서대문구 창천동 52-157 
전체 644 중 197, 주소:서울특별시 서대문구 창천동 52-32 1층 일부
전체 644 중 198, 주소:서울특별시 서대문구 창천동 30-20 
전체 644 중 199, 주소:서울특별시 서대문구 창천동 57-54번지 지하1층 
전체 644 중 200, 주소:서울특별시 서대문구 창천동 2-6
전체 644 중 201, 주소:서울특별시 서대문구 창천동 52-84번지 
전체 644 중 202, 주소:서울특별시 서대문구 창천동 53-9 지상1층 
전체 644 중 203, 주소:서울특별시 서대문구 창천동 52-36 지상1층 
전체 644 중 204, 주소:서울특별시 서대문구 창천동 9-20 지상3층
전체 644 중 205, 주소:서울특별시 서대문구 창천동 5-44번지 지하1층 전체 
전체 644 중 206, 주소:서울특별시 서대문구 창천동 53-85번지 지하1층 
전체 644 중 207, 주소:서울특별시 서대문구 창천동 52-43번지 2층 
전체 644 중 208, 주소:서울특별시 서대문구 창천동 31-8번지 
전체 644 중 209, 주소:서울특별시 서대문구 창천동 46-4번지 (지상1층) 
전체 644 중 210, 주소:서울특별시 서대문구 창천동 2-32 (지상2층)
전체 644 중 211, 주소:서울특별시 서대문구 창천동 13-77 
전체 644 중 212, 주소:서울특별시 서대문구 창천동 62-59번지 
전체 644 중 213, 주소:서울특별시 서대문구 창천동 52-94번지 (지상1층) 
전체 644 중 214, 주소:서울특별시 서대문구 창천동 52-85 
전체 644 중 215, 주소:서울특별시 서대문구 창천동 52-124번지 (지하1층) 
전체 644 중 216, 주소:서울특별시 서대문구 창천동 57-20 ,21,29,31,32호(지상1층)
전체 644 중 217, 주소:서울특별시 서대문구 창천동 57-3번지 ,42,

전체 644 중 384, 주소:서울특별시 서대문구 창천동 53-23 창서빌딩 1층 
전체 644 중 385, 주소:서울특별시 서대문구 창천동 52-91번지 2층 
전체 644 중 386, 주소:서울특별시 서대문구 창천동 53-82번지 2~3층 
전체 644 중 387, 주소:서울특별시 서대문구 창천동 33-17번지 1층 
전체 644 중 388, 주소:서울특별시 서대문구 창천동 52-11 2층
전체 644 중 389, 주소:서울특별시 서대문구 창천동 72-27번지 외 1필지 화현빌딩 1층 
전체 644 중 390, 주소:서울특별시 서대문구 창천동 30-33번지 신촌현대백화점 지하1층 
전체 644 중 391, 주소:서울특별시 서대문구 창천동 13-16 1층 우측 
전체 644 중 392, 주소:서울특별시 서대문구 창천동 510-2 쓰리알유시티아파트 B205호
전체 644 중 393, 주소:서울특별시 서대문구 창천동 30-33 10층 현대백화점신촌점
전체 644 중 394, 주소:서울특별시 서대문구 창천동 448
전체 644 중 395, 주소:서울특별시 서대문구 창천동 448번지 1층 우측 
전체 644 중 396, 주소:서울특별시 서대문구 창천동 52-94번지 2층 전부 
전체 644 중 397, 주소:서울특별시 서대문구 창천동 52-128번지 지하1층 일부 
전체 644 중 398, 주소:서울특별시 서대문구 창천동 33-9 1층 나호 
전체 644 중 399, 주소:서울특별시 서대문구 창천동 52-156 2층 전체 
전체 644 중 400, 주소:서울특별시 서대문구 창천동 2-44번지 1층 일부 
전체 644 중 401, 주소:서울특별시 서대문구 창천동 52-26 1층 일부
전체 644 중 402, 주소:서울특별시 서대문구 창천동 52-84 1층
전체 644 중 403, 주소:서울특별시 서대문구 창천동 52-155 2층 전체
전체 644 중 404, 주소:서울특별시 서대문구 창천동 31-26 1층 101호 일부
전체 644 중 405, 주소:서울특별시 서대문

전체 644 중 581, 주소:서울특별시 서대문구 창천동 57-57
전체 644 중 582, 주소:서울특별시 서대문구 창천동 53-88
전체 644 중 583, 주소:서울특별시 서대문구 창천동 31-15
전체 644 중 584, 주소:서울특별시 서대문구 창천동 52-18
전체 644 중 585, 주소:서울특별시 서대문구 창천동 30-6
전체 644 중 586, 주소:서울특별시 서대문구 창천동 31-36
전체 644 중 587, 주소:서울특별시 서대문구 창천동 52-83
전체 644 중 588, 주소:서울특별시 서대문구 창천동 52-36
전체 644 중 589, 주소:서울특별시 서대문구 창천동 52-31 지층
전체 644 중 590, 주소:서울특별시 서대문구 창천동 33-34
전체 644 중 591, 주소:서울특별시 서대문구 창천동 57-63
전체 644 중 592, 주소:서울특별시 서대문구 창천동 52-49
전체 644 중 593, 주소:서울특별시 서대문구 창천동 52-49
전체 644 중 594, 주소:서울특별시 서대문구 창천동 62-48
전체 644 중 595, 주소:서울특별시 서대문구 창천동 9-27
전체 644 중 596, 주소:서울특별시 서대문구 창천동 52-49
전체 644 중 597, 주소:서울특별시 서대문구 창천동 5-95
전체 644 중 598, 주소:서울특별시 서대문구 창천동 33-36
전체 644 중 599, 주소:서울특별시 서대문구 창천동 5-86
전체 644 중 600, 주소:서울특별시 서대문구 창천동 5-23 오시리스타워
전체 644 중 601, 주소:서울특별시 서대문구 창천동 5-64 1,2층
전체 644 중 602, 주소:서울특별시 서대문구 창천동 5-95
전체 644 중 603, 주소:서울특별시 서대문구 창천동 52-48
전체 644 중 604, 주소:서울특별시 서대문구 창천동 377 한빛빌딩
전체 644 중 605, 주소:서울특별시 서대문구 창천동 29-81 신촌르메이에르타운5
전체 644 중 606, 주소:서울특별시 서대문구 

In [11]:
naver_res

,지번주소,도로명주소,업체명,업종,URL,ID
0,서울특별시 서대문구 창천동 30-10번지,서울특별시 서대문구 신촌로 87-8 (창천동),크레페마스터,"카페,디저트",https://pcmap.place.naver.com/restaurant/19796...,19796689
1,서울특별시 서대문구 창천동 30-10번지,서울특별시 서대문구 신촌로 87-8 (창천동),신촌수제비,한식,https://pcmap.place.naver.com/restaurant/19878...,19878436
2,서울특별시 서대문구 창천동 30-10번지,서울특별시 서대문구 신촌로 87-8 (창천동),버거리 신촌점,햄버거,https://pcmap.place.naver.com/restaurant/11043...,1104306587
3,서울특별시 서대문구 창천동 13-26번지 1층,"서울특별시 서대문구 명물길 18, 1층 (창천동)",대구삼겹살,돼지고기구이,https://pcmap.place.naver.com/restaurant/31397...,31397200
4,서울특별시 서대문구 창천동 13-26번지 1층,"서울특별시 서대문구 명물길 18, 1층 (창천동)",미네르바,카페,https://pcmap.place.naver.com/restaurant/35562...,35562920
...,...,...,...,...,...,...
0,서울특별시 서대문구 창천동 57-54 2층,"서울특별시 서대문구 연세로7길 21, 2층 (창천동)",라이프롱드림 코지라운지,바(BAR),https://pcmap.place.naver.com/restaurant/12509...,1250906147
1,서울특별시 서대문구 창천동 57-54 2층,"서울특별시 서대문구 연세로7길 21, 2층 (창천동)",아무술집 신촌점,"맥주,호프",https://pcmap.place.naver.com/restaurant/19316...,1931615731
2,서울특별시 서대문구 창천동 57-54 2층,"서울특별시 서대문구 연세로7길 21, 2층 (창천동)",일웅상점,요리주점,https://pcmap.place.naver.com/restaurant/15020...,1502030404
0,서울특별시 서대문구 창천동 53-91 유림빌딩 1층,"서울특별시 서대문구 연세로7안길 42, 유림빌딩 1층 (창천동)",블랙번즈,"카페,디저트",https://pcmap.place.naver.com/restaurant/36812...,36812789


In [12]:
# 중복행 제거
naver_res_drop = naver_res.drop_duplicates(subset='ID', keep='first')

In [13]:
naver_res_drop

,지번주소,도로명주소,업체명,업종,URL,ID
0,서울특별시 서대문구 창천동 30-10번지,서울특별시 서대문구 신촌로 87-8 (창천동),크레페마스터,"카페,디저트",https://pcmap.place.naver.com/restaurant/19796...,19796689
1,서울특별시 서대문구 창천동 30-10번지,서울특별시 서대문구 신촌로 87-8 (창천동),신촌수제비,한식,https://pcmap.place.naver.com/restaurant/19878...,19878436
2,서울특별시 서대문구 창천동 30-10번지,서울특별시 서대문구 신촌로 87-8 (창천동),버거리 신촌점,햄버거,https://pcmap.place.naver.com/restaurant/11043...,1104306587
3,서울특별시 서대문구 창천동 13-26번지 1층,"서울특별시 서대문구 명물길 18, 1층 (창천동)",대구삼겹살,돼지고기구이,https://pcmap.place.naver.com/restaurant/31397...,31397200
4,서울특별시 서대문구 창천동 13-26번지 1층,"서울특별시 서대문구 명물길 18, 1층 (창천동)",미네르바,카페,https://pcmap.place.naver.com/restaurant/35562...,35562920
...,...,...,...,...,...,...
0,서울특별시 서대문구 창천동 510-10,"서울특별시 서대문구 신촌로1길 5, 1층 (창천동)",홍보석,중식당,https://pcmap.place.naver.com/restaurant/72140...,72140696
0,서울특별시 서대문구 창천동 57-50,"서울특별시 서대문구 연세로7길 15, 1,2,3층 (창천동)",시선 신촌점,이자카야,https://pcmap.place.naver.com/restaurant/16062...,1606207220
0,서울특별시 서대문구 창천동 57-13 용빌리지,"서울특별시 서대문구 연세로5나길 16, 용빌리지 1층 (창천동)",Poke all day 포케&샐러드 신촌점,"다이어트,샐러드",https://pcmap.place.naver.com/restaurant/10648...,1064836162
0,서울특별시 서대문구 창천동 52-111,"서울특별시 서대문구 연세로11길 38, 1층 (창천동)",긴자 초밥,"초밥,롤",https://pcmap.place.naver.com/restaurant/18968...,1896827351


In [14]:
# CSV 파일로 저장
naver_res_drop.to_csv('창천동.csv', index=False)